# Action Shot Extractor - Learning Guide

This notebook will guide you through understanding how the Action Shot Extractor works by exploring its components and running practical examples.

## Overview

Action Shot Extractor is an AI-powered tool that uses computer vision to identify and extract action shots from videos. It combines:
- **YOLOv8** for object detection
- **SegFormer** for image segmentation
- **OpenCV** for video processing
- **Color analysis** for filtering frames

Let's explore each component step by step!

## 1. Environment Setup

First, let's set up our environment and imports:

In [ ]:
import sys
import os

# Add the project root to Python path
project_root = os.path.abspath('../..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Import Action Shot Extractor components
from src.action_shot_extractor import (
    FrameResult, RunSummary, run_pipeline,
    hex_to_bgr, hue_distance
)

print("✅ Environment setup complete!")
print(f"Project root: {project_root}")

## 2. Understanding Color Analysis

The core of Action Shot Extractor is color-based filtering. Let's explore how it works:

In [ ]:
# Example: Converting hex colors to BGR format (used by OpenCV)
grass_green = "#228B22"
sky_blue = "#87CEEB"
jersey_red = "#FF0000"

grass_bgr = hex_to_bgr(grass_green)
sky_bgr = hex_to_bgr(sky_blue)
jersey_bgr = hex_to_bgr(jersey_red)

print(f"Grass Green: {grass_green} → BGR: {grass_bgr}")
print(f"Sky Blue: {sky_blue} → BGR: {sky_bgr}")
print(f"Jersey Red: {jersey_red} → BGR: {jersey_bgr}")

# Calculate hue distances (for color matching)
import cv2
import numpy as np

def bgr_to_hue(bgr):
    """Convert BGR color to HSV hue value."""
    bgr_array = np.uint8([[bgr]])
    hsv = cv2.cvtColor(bgr_array, cv2.COLOR_BGR2HSV)
    return hsv[0][0][0]

grass_hue = bgr_to_hue(grass_bgr)
sky_hue = bgr_to_hue(sky_bgr)

distance = hue_distance(grass_hue, sky_hue)
print(f"\nHue distance between grass and sky: {distance}°")
print(f"Grass hue: {grass_hue}°, Sky hue: {sky_hue}°")

## 3. Video Processing Basics

Let's understand how video frames are processed:

In [ ]:
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Create a sample "frame" to demonstrate processing
def create_sample_frame():
    """Create a synthetic sports-like frame for demonstration."""
    frame = np.zeros((480, 640, 3), dtype=np.uint8)
    
    # Green field background
    frame[:, :] = [34, 139, 34]  # Forest green in BGR
    
    # Add some "players" (colored rectangles)
    cv2.rectangle(frame, (100, 200), (150, 300), (0, 0, 255), -1)  # Red player
    cv2.rectangle(frame, (300, 180), (350, 280), (255, 255, 255), -1)  # White player
    cv2.rectangle(frame, (500, 220), (550, 320), (0, 255, 255), -1)  # Yellow player
    
    # Add a "ball" (white circle)
    cv2.circle(frame, (320, 240), 15, (255, 255, 255), -1)
    
    return frame

sample_frame = create_sample_frame()

# Display the sample frame
plt.figure(figsize=(10, 6))
plt.imshow(cv2.cvtColor(sample_frame, cv2.COLOR_BGR2RGB))
plt.title('Sample Sports Frame')
plt.axis('off')
plt.show()

print(f"Frame shape: {sample_frame.shape}")
print(f"Frame dtype: {sample_frame.dtype}")

## 4. Color-Based Frame Analysis

Let's analyze the color composition of our sample frame:

In [ ]:
def analyze_frame_colors(frame, target_color_bgr, tolerance=30):
    """Analyze how much of a frame contains a target color."""
    # Convert to HSV for better color matching
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    target_hsv = cv2.cvtColor(np.uint8([[target_color_bgr]]), cv2.COLOR_BGR2HSV)[0][0]
    
    # Create color mask
    lower = np.array([max(0, target_hsv[0] - tolerance), 50, 50])
    upper = np.array([min(179, target_hsv[0] + tolerance), 255, 255])
    mask = cv2.inRange(hsv_frame, lower, upper)
    
    # Calculate percentage
    total_pixels = frame.shape[0] * frame.shape[1]
    matching_pixels = cv2.countNonZero(mask)
    percentage = (matching_pixels / total_pixels) * 100
    
    return percentage, mask

# Analyze green (field) content
green_percentage, green_mask = analyze_frame_colors(sample_frame, [34, 139, 34])

# Analyze red (player) content
red_percentage, red_mask = analyze_frame_colors(sample_frame, [0, 0, 255])

print(f"Green (field) coverage: {green_percentage:.1f}%")
print(f"Red (player) coverage: {red_percentage:.1f}%")

# Visualize masks
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(cv2.cvtColor(sample_frame, cv2.COLOR_BGR2RGB))
axes[0].set_title('Original Frame')
axes[0].axis('off')

axes[1].imshow(green_mask, cmap='gray')
axes[1].set_title(f'Green Mask ({green_percentage:.1f}%)')
axes[1].axis('off')

axes[2].imshow(red_mask, cmap='gray')
axes[2].set_title(f'Red Mask ({red_percentage:.1f}%)')
axes[2].axis('off')

plt.tight_layout()
plt.show()

## 5. Understanding the Pipeline Components

Let's explore the main components of the Action Shot Extractor pipeline:

In [ ]:
# Read and understand the pipeline structure
from src.action_shot_extractor.pipeline import FrameResult, RunSummary

# Create a sample FrameResult
sample_result = FrameResult(
    frame_number=150,
    timestamp=5.0,  # 5 seconds into video
    sharpness_score=0.85,
    color_match_score=0.92,
    objects_detected=['person', 'sports ball'],
    confidence_scores=[0.89, 0.76],
    selected=True,
    reason="High action content with clear objects"
)

print("Sample FrameResult:")
print(f"  Frame: {sample_result.frame_number} at {sample_result.timestamp}s")
print(f"  Sharpness: {sample_result.sharpness_score:.2f}")
print(f"  Color match: {sample_result.color_match_score:.2f}")
print(f"  Objects: {sample_result.objects_detected}")
print(f"  Confidences: {sample_result.confidence_scores}")
print(f"  Selected: {sample_result.selected}")
print(f"  Reason: {sample_result.reason}")

## 6. Sharpness Detection

Sharpness is crucial for identifying clear action shots:

In [ ]:
def calculate_sharpness(frame):
    """Calculate frame sharpness using Laplacian variance."""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    laplacian = cv2.Laplacian(gray, cv2.CV_64F)
    variance = laplacian.var()
    return variance

def create_blurred_frame(frame, blur_amount):
    """Create a blurred version of the frame."""
    return cv2.GaussianBlur(frame, (blur_amount, blur_amount), 0)

# Test sharpness on different blur levels
blur_levels = [1, 15, 31, 51]
sharpness_scores = []

fig, axes = plt.subplots(2, len(blur_levels), figsize=(16, 8))

for i, blur in enumerate(blur_levels):
    if blur == 1:
        test_frame = sample_frame.copy()
        title = "Original"
    else:
        test_frame = create_blurred_frame(sample_frame, blur)
        title = f"Blur {blur}px"
    
    sharpness = calculate_sharpness(test_frame)
    sharpness_scores.append(sharpness)
    
    # Display frame
    axes[0, i].imshow(cv2.cvtColor(test_frame, cv2.COLOR_BGR2RGB))
    axes[0, i].set_title(f"{title}\nSharpness: {sharpness:.0f}")
    axes[0, i].axis('off')
    
    # Display edge detection
    gray = cv2.cvtColor(test_frame, cv2.COLOR_BGR2GRAY)
    edges = cv2.Laplacian(gray, cv2.CV_64F)
    axes[1, i].imshow(np.abs(edges), cmap='gray')
    axes[1, i].set_title('Edge Detection')
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()

# Plot sharpness trend
plt.figure(figsize=(10, 6))
plt.plot(blur_levels, sharpness_scores, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Blur Level (pixels)')
plt.ylabel('Sharpness Score')
plt.title('Sharpness vs Blur Level')
plt.grid(True, alpha=0.3)
plt.show()

print("Observation: Sharpness score decreases as blur increases")
print("This helps identify clear, action-worthy frames!")

## 7. Understanding the CLI Interface

Let's explore the command-line interface structure:

In [ ]:
# Read the CLI module to understand available options
cli_path = os.path.join(project_root, 'src', 'action_shot_extractor', 'cli.py')

print("CLI Command Structure:")
print("action-shot-extractor [OPTIONS] VIDEO_PATH")
print()
print("Key Options:")
print("  --colors TEXT        Target colors (hex format, comma-separated)")
print("  --output-dir PATH    Output directory for extracted frames")
print("  --max-frames INT     Maximum frames to extract")
print("  --min-sharpness FLOAT Minimum sharpness threshold")
print("  --color-tolerance INT Color matching tolerance")
print("  --fps-sample INT     Sample every N frames for performance")
print()
print("Example usage:")
print('action-shot-extractor --colors "#00FF00,#FF0000" --max-frames 10 video.mp4')

## 8. Putting It All Together

Let's simulate a complete pipeline run:

In [ ]:
def simulate_pipeline_run():
    """Simulate a complete Action Shot Extractor pipeline run."""
    
    # Simulate processing multiple frames
    frames_data = [
        (50, 1.67, 0.65, 0.45, ['person'], [0.75], False, "Low sharpness"),
        (150, 5.0, 0.85, 0.92, ['person', 'sports ball'], [0.89, 0.76], True, "High action content"),
        (300, 10.0, 0.55, 0.88, ['person'], [0.82], False, "Motion blur detected"),
        (450, 15.0, 0.91, 0.87, ['person', 'person'], [0.91, 0.85], True, "Multiple players detected"),
        (600, 20.0, 0.78, 0.23, ['person'], [0.66], False, "Poor color match"),
    ]
    
    results = []
    for data in frames_data:
        result = FrameResult(
            frame_number=data[0],
            timestamp=data[1],
            sharpness_score=data[2],
            color_match_score=data[3],
            objects_detected=data[4],
            confidence_scores=data[5],
            selected=data[6],
            reason=data[7]
        )
        results.append(result)
    
    # Create summary
    selected_frames = [r for r in results if r.selected]
    
    summary = RunSummary(
        total_frames=len(results),
        processed_frames=len(results),
        selected_frames=len(selected_frames),
        avg_sharpness=sum(r.sharpness_score for r in results) / len(results),
        avg_color_match=sum(r.color_match_score for r in results) / len(results),
        processing_time=12.5,
        output_directory="./output/"
    )
    
    return results, summary

# Run simulation
results, summary = simulate_pipeline_run()

print("🎬 Pipeline Simulation Results")
print("=" * 40)
print(f"Total frames processed: {summary.total_frames}")
print(f"Frames selected: {summary.selected_frames}")
print(f"Selection rate: {(summary.selected_frames/summary.total_frames)*100:.1f}%")
print(f"Average sharpness: {summary.avg_sharpness:.2f}")
print(f"Average color match: {summary.avg_color_match:.2f}")
print(f"Processing time: {summary.processing_time:.1f}s")
print()

print("Selected Frames:")
for result in results:
    if result.selected:
        print(f"  Frame {result.frame_number} ({result.timestamp}s): {result.reason}")

# Visualize results
frame_numbers = [r.frame_number for r in results]
sharpness_scores = [r.sharpness_score for r in results]
color_scores = [r.color_match_score for r in results]
selected = [r.selected for r in results]

plt.figure(figsize=(12, 8))

# Plot sharpness and color scores
plt.subplot(2, 1, 1)
plt.plot(frame_numbers, sharpness_scores, 'b-o', label='Sharpness', alpha=0.7)
plt.plot(frame_numbers, color_scores, 'g-s', label='Color Match', alpha=0.7)

# Highlight selected frames
selected_frames = [frame_numbers[i] for i, sel in enumerate(selected) if sel]
selected_sharpness = [sharpness_scores[i] for i, sel in enumerate(selected) if sel]
plt.scatter(selected_frames, selected_sharpness, c='red', s=100, marker='*', 
           label='Selected Frames', zorder=5)

plt.xlabel('Frame Number')
plt.ylabel('Score')
plt.title('Frame Analysis Results')
plt.legend()
plt.grid(True, alpha=0.3)

# Show selection timeline
plt.subplot(2, 1, 2)
selection_y = [1 if sel else 0 for sel in selected]
plt.scatter(frame_numbers, selection_y, c=['red' if sel else 'gray' for sel in selected], 
           s=[100 if sel else 50 for sel in selected], alpha=0.7)
plt.xlabel('Frame Number')
plt.ylabel('Selected')
plt.title('Frame Selection Timeline')
plt.yticks([0, 1], ['No', 'Yes'])
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Next Steps and Exercises

Now that you understand the components, try these exercises:

### Exercise 1: Color Experimentation
Modify the color analysis function to work with different sports:
- Basketball (orange ball, wooden court)
- Tennis (green court, yellow ball)
- Soccer (green field, white ball)

### Exercise 2: Custom Sharpness Metrics
Implement alternative sharpness detection methods:
- Sobel edge detection
- Variance of gradients
- FFT-based methods

### Exercise 3: Pipeline Optimization
Think about how to optimize the pipeline:
- Skip frames vs. process all frames
- Multi-threading for faster processing
- GPU acceleration for AI models

### Exercise 4: Real Video Testing
Test the tool with actual video files:
```bash
# Install and test
cd ../.. 
poetry install
action-shot-extractor --help
```

## Conclusion

You've now learned:
1. ✅ How color analysis works in sports video processing
2. ✅ How sharpness detection identifies clear frames
3. ✅ How the pipeline combines multiple metrics for frame selection
4. ✅ How to interpret and visualize processing results

The Action Shot Extractor combines computer vision, AI object detection, and traditional image processing to automatically find the best moments in sports videos. Each component serves a specific purpose in creating a robust action shot detection system.

**Happy learning! 🚀**